In [2]:
! pip install spacy

  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/92/fb/d1f0605e1e8627226c6c96053fe1632e9a04a3fbcd8b5d715528cb95eb97/spacy-3.7.4-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for spacy-legacy<3.1.0,>=3.0.11 from https://files.pythonhosted.org/packages/c3/55/12e842c70ff8828e34e543a2c7176dac4da006ca6901c9e8b43efab8bc6b/spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/71/46/af01a20ec368bd9cb49a1d2df15e3eca113bbf6952cc1f2a47f1c6801a7f/murmurhash-1.0.10-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for cymem<2.1.0,>=2.0.2 from https://files.pythonhosted.org/packages/c1/c3/dd

In [3]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment=None

In [13]:
full_df=pd.read_csv("twcs_v1.csv",nrows=3000)
df=full_df[["text"]]
df["text"]=df["text"].astype(str);
full_df

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
...,...,...,...,...,...,...,...
2995,3975,116663,True,Wed Nov 01 21:02:24 +0000 2017,@British_Airways The staff in the counter issu...,"3,97,73,978",3974.0
2996,3977,British_Airways,False,Thu Nov 02 08:47:23 +0000 2017,@116663 contact telephone number. We'll look a...,NaN,3975.0
2997,3978,British_Airways,False,Thu Nov 02 08:46:39 +0000 2017,@116663 Hi Omar. Please can you DM more inform...,NaN,3975.0
2998,3976,116663,True,Tue Oct 31 16:33:33 +0000 2017,"@British_Airways i hope you can help me, becus...",3974,NaN


##### Lower Casing

In [14]:
df["text_lower"]=df["text"].str.lower()

In [15]:
df.head()

,text,text_lower
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...
4,@sprintcare I did.,@sprintcare i did.


#### Removal of Punctuations

One another common text preprocessing technique is to remove the punctuations from the text data. This is again a text standardization process that will help to treat 'hurray' and 'hurray!' in the same way.

We also need to carefully choose the list of punctuations to exclude depending on the use case. For example, the string.punctuation in python contains the following punctuation symbols

!"#$%&'()*+,-./:;<=>?@[\]^_{|}~`

We can add or remove more punctuations as per our need.

In [38]:
PUNCT_TO_REMOVE=string.punctuation
def remove_punctuation(text):
    "custom function to remove the punctuation"
    return text.translate(str.maketrans('','',PUNCT_TO_REMOVE))

In [39]:
df["text_to_punct"]=df["text_lower"].apply(lambda text:remove_punctuation(text));

In [40]:
df.head()

,text,text_lower,text_to_punct
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did


#### Removal of stopwords

Stopwords are commonly occuring words in a language like 'the', 'a' and so on. They can be removed from the text most of the times, as they don't provide valuable information for downstream analysis. In cases like Part of Speech tagging, we should not remove them as provide very valuable information about the POS.

These stopword lists are already compiled for different languages and we can safely use them. For example, the stopword list for english language from the nltk package can be seen below.

In [46]:
from nltk.corpus import stopwords
stopWords=stopwords.words('english')

In [54]:
def remove_stopword(text):
    "function of removing stop words"
    return " ".join([word for word in str(text).split() if word not in stopWords])

In [55]:
df["remove_stopwords"]=df["text_to_punct"].apply(lambda text:remove_stopword(text));

In [56]:
df.head()

,text,text_lower,text_to_punct,remove_stopwords
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare


#### Removal of Frequent words

In the previos preprocessing step, we removed the stopwords based on language information. But say, if we have a domain specific corpus, we might also have some frequent words which are of not so much importance to us.

So this step is to remove the frequent words in the given corpus. If we use something like tfidf, this is automatically taken care of.

Let us get the most common words and then remove them in the next step

In [58]:
from collections import Counter
cnt=Counter()
for text in df["remove_stopwords"].values:
    for word in text.split():
        cnt[word]+=1;

In [60]:
cnt.most_common(10)

[('us', 461),
 ('please', 388),
 ('dm', 303),
 ('help', 302),
 ('thanks', 230),
 ('hi', 205),
 ('get', 169),
 ('know', 158),
 ('like', 152),
 ('sorry', 152)]

In [61]:
FREQWORDS=set([w for (w,wc) in cnt.most_common(10)])
def remove_freq_word(text):
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])
 

In [62]:
df["reomve_freqword"]=df["remove_stopwords"].apply(lambda text:remove_freq_word(text))

In [63]:
df.head()

,text,text_lower,text_to_punct,remove_stopwords,reomve_freqword
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would assist would need priv...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click ‘mess...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare


##### Removal of Rare words

This is very similar to previous preprocessing step but we will remove the rare words from the corpus.

In [69]:
n_rare_words=10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rare_word(text):
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

In [72]:
df["remove_rarewords"]=df["reomve_freqword"].apply(lambda text:remove_rare_word(text))

In [73]:
df.head()

,text,text_lower,text_to_punct,remove_stopwords,reomve_freqword,remove_rarewords
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would assist would need priv...,115712 understand would assist would need priv...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click ‘mess...,115712 send private message assist click ‘mess...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare,sprintcare


#### Stemming

Stemming is the process of reducing inflected (or sometimes derived) words to their word stem, base or root form

For example, if there are two words in the corpus walks and walking, then stemming will stem the suffix to make them walk. But say in another example, we have two words console and consoling, the stemmer will remove the suffix and make them consol which is not a proper english word.

In [74]:
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

In [75]:
df["text_stemmed"]=df["remove_rarewords"].apply(lambda text:stem_words(text))

In [76]:
df.head()

,text,text_lower,text_to_punct,remove_stopwords,reomve_freqword,remove_rarewords,text_stemmed
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would assist would need priv...,115712 understand would assist would need priv...,115712 understand would assist would need priv...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose,sprintcare propose,sprintcar propos
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcar sent sever privat messag one respond...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click ‘mess...,115712 send private message assist click ‘mess...,115712 send privat messag assist click ‘messag...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare,sprintcare,sprintcar


We can see that words like private and propose have their e at the end chopped off due to stemming. This is not intented. What can we do fort hat? We can use Lemmatization in such cases.

Also this porter stemmer is for English language. If we are working with other languages, we can use snowball stemmer. The supported languages for snowball stemmer are

In [77]:
from nltk.stem.snowball import SnowballStemmer
SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

#### Lemmatization

Lemmatization is similar to stemming in reducing inflected words to their word stem but differs in the way that it makes sure the root word (also called as lemma) belongs to the language.

As a result, this one is generally slower than stemming process. So depending on the speed requirement, we can choose to use either stemming or lemmatization.

Let us use the WordNetLemmatizer in nltk to lemmatize our sentences

In [87]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [88]:
df["text_lemmatized"]=df["remove_rarewords"].apply(lambda text:lemmatize_words(text))

In [89]:
df.head()

,text,text_lower,text_to_punct,remove_stopwords,reomve_freqword,remove_rarewords,text_stemmed,text_lemmatized
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would assist would need priv...,115712 understand would assist would need priv...,115712 understand would assist would need priv...,115712 understand would assist would need priv...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose,sprintcare propose,sprintcar propos,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcar sent sever privat messag one respond...,sprintcare sent several private message one re...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click ‘mess...,115712 send private message assist click ‘mess...,115712 send privat messag assist click ‘messag...,115712 send private message assist click ‘mess...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare,sprintcare,sprintcar,sprintcare


### Removal of Emojis

With more and more usage of social media platforms, there is an explosion in the usage of emojis in our day to day life as well. Probably we might need to remove these emojis for some of our textual analysis.